In [50]:
import tensorflow as tf 
from tensorflow.keras import datasets,layers,optimizers,Sequential,metrics
from tensorflow import keras
import os 

In [63]:
def preprocess(x,y):
    x = tf.cast(x,dtype = tf.float32)/255. - 1. #归一化到[-1,1]区间内
    y = tf.cast(y,dtype = tf.int32)
    return x,y

In [70]:
batchsize = 128
(x,y),(x_val,y_val) = datasets.cifar10.load_data()
y = tf.squeeze(y) # 缩减冗余的维度
y_val=tf.squeeze(y_val)

y = tf.one_hot(y,depth=10)
y_val = tf.one_hot(y_val,depth=10)
print(x.shape,y.shape,x_val.shape,y_val.shape, x.max())

(50000, 32, 32, 3) (50000, 10) (10000, 32, 32, 3) (10000, 10) 255


In [71]:
train_db = tf.data.Dataset.from_tensor_slices((x,y))
train_db = train_db.map(preprocess).shuffle(10000).batch(batchsize) # 进行预处理

test_db = tf.data.Dataset.from_tensor_slices((x_val,y_val))
test_db = test_db.map(preprocess).batch(batchsize) # 进行预处理

In [72]:
# 测试一个batch的大小
sample = next(iter(train_db))
print(sample[0].shape,sample[1].shape)

(128, 32, 32, 3) (128, 10)


In [73]:
# 自定义层
class MyDense(layers.Layer):
    # 自定义网络层, 代替现有的Layer的子类
    def __init__(self,input_dim,output_dim):
        super(MyDense,self).__init__()
        self.kernel = self.add_variable('w',[input_dim,output_dim]) # Layer自有的添加元素的方法, 必须用这个方法
#         self.bias = self.add_variable('b',[output_dim]) # 我们这里不适用bias
        
        
    
    def __call__(self,inputs,training = None):
        x = inputs@self.kernel # 不加bias直接输出
        return x

In [74]:
# 自定义网络
class MyNetwork(keras.Model):
    def __init__(self):
        super(MyNetwork,self).__init__()
        self.fc1 = MyDense(32*32*3,512)  # 定义第一层函数
        self.fc2 = MyDense(512,256)# 定义第二层函数
        self.fc3 = MyDense(256,128)# 定义第三层函数
        self.fc4 = MyDense(128,64)# 定义第四层函数
        self.fc5 = MyDense(64,32)# 定义第五层函数
        self.fc6 = MyDense(32,10)# 定义第五层函数
    
    def __call__(self,inputs,training = None):
        x = tf.reshape(inputs,[-1,32*32*3])
        x = self.fc1(x)
        x = tf.nn.relu(x)
        x = self.fc2(x)
        x = tf.nn.relu(x)
        x = self.fc3(x)
        x = tf.nn.relu(x)
        x = self.fc4(x)
        x = tf.nn.relu(x)
        x = self.fc5(x)
        x = tf.nn.relu(x)
        x = self.fc6(x)
        return x

In [75]:
model = MyNetwork()
model.compile(optimizer = optimizers.Adam(learning_rate = 1e-4),
             loss = tf.losses.CategoricalCrossentropy(from_logits = True),
             metrics = ['accuracy'])

In [77]:
model.fit(train_db,epochs=30,validation_data=test_db,validation_freq=1) # validation_freq=代表没一个epoch训练完成后进行一次验证集测试

Epoch 1/30
391/391 [==============================] - 4s 11ms/step - loss: 1.2997 - accuracy: 0.5403 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/30
391/391 [==============================] - 4s 9ms/step - loss: 1.2810 - accuracy: 0.5495 - val_loss: 1.3983 - val_accuracy: 0.5052
Epoch 3/30
391/391 [==============================] - 4s 10ms/step - loss: 1.2582 - accuracy: 0.5556 - val_loss: 1.3722 - val_accuracy: 0.5127
Epoch 4/30
391/391 [==============================] - 4s 11ms/step - loss: 1.2448 - accuracy: 0.5630 - val_loss: 1.3906 - val_accuracy: 0.5074
Epoch 5/30
391/391 [==============================] - 4s 11ms/step - loss: 1.2306 - accuracy: 0.5646 - val_loss: 1.3627 - val_accuracy: 0.5185
Epoch 6/30
391/391 [==============================] - 4s 11ms/step - loss: 1.2089 - accuracy: 0.5738 - val_loss: 1.3959 - val_accuracy: 0.5016
Epoch 7/30
391/391 [==============================] - 4s 10ms/step - loss: 1.1952 - accuracy: 0.5789 - val_loss: 1.3711 - val_accuracy:

In [78]:
model.evaluate(test_db)
model.save_weights('./ckpt/2019-11-13.ckpt')
del model
# 保存模型的参数

79/79 [==============================] - 0s 6ms/step - loss: 1.4510 - accuracy: 0.5288


In [79]:
model_new = MyNetwork()
model_new.load_weights('./ckpt/2019-11-13.ckpt')
# 加载模型参数后如果项进行测试, 还需要先对模型进行编译compile,设定优化其,衡量指标, 损失函数等信息
model_new.compile(optimizer = optimizers.Adam(learning_rate = 1e-4),
                 loss = tf.losses.CategoricalCrossentropy(from_logits = True),
                 metrics = ['accuracy'])
model_new.evaluate(test_db)
# 加载模型参数, 未经训练的模型直接使用之前的参数直接获得了0.484的准确率

79/79 [==============================] - 0s 6ms/step - loss: 1.4510 - accuracy: 0.5288


[1.4510230761540086, 0.5288]